In [5]:
import os

import numpy as np
import trimesh
from tqdm import tqdm

# /cvlabdata2/home/remelli/DeepSDF/bin

In [50]:
root = '/cvlabdata1/home/remelli/SVR/ShapeNetCore.v2/02958343/'
save_dir = '/cvlabdata2/home/artem/Data/cars_refined'

objects = list()
for (dirpath, dirnames, filenames) in os.walk(root):
    objects += [os.path.join(dirpath, file) for file in filenames if file == 'isosurf.obj']




835it [00:11, 385.69it/s]

566it [00:16, 232.17it/s]

In [6]:
objects = ['/cvlabdata2/home/artem/Expirements/SavedTransforms/DeepSDF-CFD8-car3-lift-min/meshes/00010.ply',
           '/cvlabdata2/home/artem/Expirements/SavedTransforms/DeepSDF-CFD8-car3-lift-min/meshes/00000.ply']
save_dir = '/cvlabdata2/home/artem/Expirements/SavedTransforms/DeepSDF-CFD8-car3-lift-min/meshes_for_simulations'

In [12]:
rotation_transform = np.array([[-1,  0,  0,  0], 
                               [ 0,  1,  0,  0],
                               [ 0,  0, -1,  0],
                               [ 0,  0,  0,  1]], dtype=np.float)

for obj in objects:
    mesh = trimesh.load(obj)
    car_name = obj.split('/')[-1][:-4]
    mc = mesh.center_mass
    translation_matrix = np.array([[ 1,  0,  0,  -mc[0]], 
                                   [ 0,  1,  0,  -mc[1]],
                                   [ 0,  0,  1,  -mc[2]],
                                   [ 0,  0,  0,  1]], dtype=np.float)
    
    width = mesh.bounds[1][0] - mesh.bounds[0][0]
    sf = 1 / width
    scaling_matrix = np.array([[sf,  0,  0,  0], 
                               [ 0, sf,  0,  0],
                               [ 0,  0, sf,  0],
                               [ 0,  0,  0,  1]], dtype=np.float)

    final_matrix = np.matmul(np.matmul(rotation_transform, translation_matrix), scaling_matrix)
    mesh.apply_transform(final_matrix)

    np.save(os.path.join( os.path.join(save_dir, 'transforms'), car_name + '.npy'), final_matrix)
    mesh.export(os.path.join( os.path.join(save_dir, 'cars'), car_name + '.stl') )

In [20]:
reference_mesh_path = '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/stl/0005.stl'
reference_mesh = trimesh.load(reference_mesh_path)

In [27]:
reference_mesh.bounds

array([[-0.5232693 , -0.12063155, -0.21424496],
       [ 0.47672951,  0.14396769,  0.21383004]])

In [38]:
import json
import os
import numpy as np

In [59]:
files = list(filter(lambda x: x[-4:] == ".npz",  
                    os.listdir("data/SdfSamples/ShapeNetV2/02958343")))[:1500]
files = list(map(lambda x: x[:-4], files))
np.random.shuffle(files)

In [60]:
train_set = {"ShapeNetV2" : {"02958343" : files[:-64]} }
test_set = {"ShapeNetV2" : {"02958343" : files[-64:]} }

In [61]:
with open("examples/splits/sv2_cars_test.json", 'w') as file: 
    json.dump(test_set, file)
with open("examples/splits/sv2_cars_train.json", 'w') as file: 
    json.dump(train_set, file)